In [32]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests

from config import weather_api_key

In [33]:
latlong_df = pd.read_csv("winery_latlong_data.csv")
latlong_df.head()

,Unnamed: 0,winery_search_term,latitude,longitude,number_of_results,status
0,0,Nicosia winery Italy,37.622150,15.074750,1,OK
1,1,Quinta dos Avidagos winery Portugal,41.183402,-7.757935,1,OK
2,2,NaN,NaN,NaN,0,ZERO_RESULTS
3,3,St. Julian winery US,42.212251,-85.891713,1,OK
4,4,Sweet Cheeks winery US,43.956670,-123.279135,1,OK


In [34]:
latlong_df.dropna(inplace=True)

In [35]:
# Create lists of latitudes and longitudes
lats = list(latlong_df['latitude'])
lngs = list(latlong_df['longitude'])

wineries = list(latlong_df['winery_search_term'])

In [36]:
# Starting URL for Weather Map API Call.
url = "http://history.openweathermap.org/data/2.5/aggregated/month?units=imperial&appid=" + weather_api_key

In [37]:
months = ["1","2","3","4","5","6","7","8","9","10","11","12"]
all_winery_weather_data = []

def get_avg_temp (winery, winery_count):
    winery_temp_data  = []
    for month in months:
        # Create endpoint URL with each winery.
        winery_url = url + "&lat=" + str(lats[winery_count]) + "&lon=" + str(lngs[winery_count]) + "&month=" + month
        
        # Parse the JSON and retrieve data.
        winery_weather = requests.get(winery_url).json()
        # Parse out the needed data.
        month_avg_temp = winery_weather["result"]["temp"]["mean"]   
        winery_temp_data.append(month_avg_temp)
    
    #calculate avg temp for year
    avg_temp = sum(winery_temp_data) / len(winery_temp_data)
    
    return avg_temp

In [38]:
def get_avg_pressure (winery, winery_count):
    winery_pressure_data  = []
    for month in months:
        # Create endpoint URL with each winery.
        winery_url = url + "&lat=" + str(lats[winery_count]) + "&lon=" + str(lngs[winery_count]) + "&month=" + month
        
        # Parse the JSON and retrieve data.
        winery_weather = requests.get(winery_url).json()
        # Parse out the needed data.
        month_avg_pressure = winery_weather["result"]["pressure"]["mean"]
        winery_pressure_data.append(month_avg_pressure)
        
    #calculate avg pressure for year
    avg_pressure = sum(winery_pressure_data) / len(winery_pressure_data)
    
    return avg_pressure

In [39]:
def get_avg_humidity (winery, winery_count):
    winery_humidity_data  = []
    for month in months:
        # Create endpoint URL with each winery.
        winery_url = url + "&lat=" + str(lats[winery_count]) + "&lon=" + str(lngs[winery_count]) + "&month=" + month
        
        # Parse the JSON and retrieve data.
        winery_weather = requests.get(winery_url).json()
        # Parse out the needed data.
        month_avg_humidity = winery_weather["result"]["humidity"]["mean"]
        winery_humidity_data.append(month_avg_humidity)
        
    #calculate avg humidity for year
    avg_humidity = sum(winery_humidity_data) / len(winery_humidity_data)
    
    return avg_humidity

In [40]:
def get_avg_precip (winery, winery_count):
    winery_precip_data  = []
    for month in months:
        # Create endpoint URL with each winery.
        winery_url = url + "&lat=" + str(lats[winery_count]) + "&lon=" + str(lngs[winery_count]) + "&month=" + month
        
        # Parse the JSON and retrieve data.
        winery_weather = requests.get(winery_url).json()
        # Parse out the needed data.
        month_avg_precip = winery_weather["result"]["precipitation"]["mean"]
            
        winery_precip_data.append(month_avg_precip)
    
    #calculate avg temp for year
    avg_precip = sum(winery_precip_data) / len(winery_precip_data)
    
    return avg_precip

In [41]:
# Create an empty list to hold the weather data.
winery_weather_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1
winery_count = 0

# Loop through all the wineries in the list.
for i, winery in enumerate(wineries):

    # Group wineries in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {winery}")
    # Add 1 to the record count and winery count.
    record_count += 1
    
    # Run an API request for each of the wineries.
    try:
        # Parse the JSON and retrieve data.
        avg_temp = get_avg_temp(winery, winery_count)
        avg_pressure = get_avg_pressure(winery, winery_count)
        avg_humidity = get_avg_humidity(winery, winery_count)
        avg_precip = get_avg_precip(winery, winery_count)
               
        # Append the winery information into city_data list.
        winery_weather_data.append({"Winery": wineries[winery_count],
                          "Average Temperature (Kelvin)": avg_temp,
                          "Average Air Pressure (hPa)": avg_pressure,
                          "Average Humidity (%)": avg_humidity,
                          "Average Precipitation (mm)": avg_precip})

    # If an error is experienced, skip the winery.
    except:
        print("Weather not found. Skipping...")
        pass
    
    # Add to winery count
    winery_count += 1

    # Indicate that Data Loading is complete.
    print("-----------------------------")
    print("Data Retrieval Complete      ")
    print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | Nicosia winery Italy
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 2 of Set 1 | Quinta dos Avidagos winery Portugal
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 3 of Set 1 | St. Julian winery US
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 4 of Set 1 | Sweet Cheeks winery US
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 5 of Set 1 | Tandem winery Spain
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 6 of Set 1 | Terre di Giurfo winery Italy
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 7 of Set 1 | Trimbach winery France
----------

In [42]:
# Convert the array of dictionaries to a Pandas DataFrame.
winery_weather_data_df = pd.DataFrame(winery_weather_data)
winery_weather_data_df

,Winery,Average Temperature (Kelvin),Average Air Pressure (hPa),Average Humidity (%),Average Precipitation (mm)
0,Nicosia winery Italy,290.745000,1011.390000,70.790833,0.049167
1,Quinta dos Avidagos winery Portugal,289.772500,1016.296667,76.410000,0.095000
2,St. Julian winery US,283.365000,1016.224167,73.552500,0.118333
3,Sweet Cheeks winery US,284.865000,1016.988333,75.646667,0.109167
4,Tandem winery Spain,286.504167,1004.892500,75.960833,0.093333
5,Terre di Giurfo winery Italy,291.126667,1015.330833,72.390833,0.049167
6,Trimbach winery France,284.305833,1013.710833,76.263333,0.119167
7,Heinz Eifel winery Germany,290.584167,1018.370833,72.263333,0.125000
8,Jean-Baptiste Adam winery France,284.305833,1013.710833,76.263333,0.119167
